In [ ]:
import torch
from google.colab import drive
drive.mount('/gdrive/')
!ls /gdrive
import os
import torch.nn as nn
!pip install transformers
from transformers import RobertaTokenizer

BASE_PATH = '/gdrive/My Drive/Embeddings/'
if not os.path.exists(BASE_PATH):
    os.makedirs(BASE_PATH)
os.chdir(BASE_PATH)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


Drive already mounted at /gdrive/; to attempt to forcibly remount, call drive.mount("/gdrive/", force_remount=True).
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
MyDrive
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.


In [ ]:
import torch.nn.functional as F

class LSTMModel(nn.Module):
    def __init__(self):
        super().__init__()
        embedding_dim = 100
        self.embedding = nn.Embedding(num_embeddings=50257, embedding_dim=embedding_dim, padding_idx=1)


        self.hidden = 512
        self.orig = nn.LSTM(input_size=embedding_dim, hidden_size=self.hidden, batch_first=True)
        self.edit = nn.LSTM(input_size=embedding_dim, hidden_size=self.hidden, batch_first=True)


        self.fc1 = nn.Linear(self.hidden, self.hidden//2)
        self.fc2 = nn.Linear(self.hidden//2, self.hidden//4)
        self.fc3 = nn.Linear(self.hidden//4, 1)

        self.d1 = nn.Dropout(0.3)
        self.d2 = nn.Dropout(0.3)
        self.d3 = nn.Dropout(0.3)
        self.d4 = nn.Dropout(0.3)

    
    def forward(self, x, x_edited):
      x = self.embedding(x)  #[32, 28, 64]
      x_edited = self.embedding(x_edited)

      x, h = self.orig(x) #[32, 28, 128]
      x_edited, h = self.edit(x_edited)
      x = F.leaky_relu(self.d1(x))
      x_edited = F.leaky_relu(self.d2(x_edited))

      x = x[:,-1,:]
      x_edited = x_edited[:,-1,:]
      # input = torch.cat([x, x_edited], dim=1).reshape(-1, 2*self.hidden)
      input = torch.sub(x, x_edited)
      input = F.leaky_relu(self.d3(self.fc1(input)))  
      input =  F.leaky_relu(self.d3(self.fc2(input)))
      input = self.fc3(input)
      return input
tokenizer_roberta = RobertaTokenizer.from_pretrained('roberta-base')

def tokenize(sentence, tokenizer):
    tokens = tokenizer(sentence, return_tensors='pt')
    return tokens


In [14]:
m = torch.load( "./model")
m.to(device)
m.eval()
headline = "Kushner to visit Mexico following latest Trump tirades"
changed = "Kushner to visit therapist following latest Trump tirades"
org = tokenize(headline, tokenizer_roberta)['input_ids'][0]
edit = tokenize(changed, tokenizer_roberta)['input_ids'][0]
org = org.unsqueeze(0)
edit = edit.unsqueeze(0)
m(org.to(device), edit.to(device))




tensor([[0.5848]], device='cuda:0', grad_fn=<AddmmBackward>)

In [15]:
headline = "As Rhetoric Escalates , What Do We know About North Korea 's Nuclear Arsenal ?"
changed = "As Rhetoric Escalates , What Do We care About North Korea 's Nuclear Arsenal ?"
org = tokenize(headline, tokenizer_roberta)['input_ids'][0]
edit = tokenize(changed, tokenizer_roberta)['input_ids'][0]
org = org.unsqueeze(0)
edit = edit.unsqueeze(0)
m(org.to(device), edit.to(device))


tensor([[0.8115]], device='cuda:0', grad_fn=<AddmmBackward>)

In [16]:
headline = "Patients die as lawyers ransack Pakistan hospital"
changed = "Patients die as poodles ransack Pakistan hospital"
org = tokenize(headline, tokenizer_roberta)['input_ids'][0]
edit = tokenize(changed, tokenizer_roberta)['input_ids'][0]
org = org.unsqueeze(0)
edit = edit.unsqueeze(0)
m(org.to(device), edit.to(device))


tensor([[0.5490]], device='cuda:0', grad_fn=<AddmmBackward>)